## Atenção! Você precisa de Python 3.6+ para esta apresentação funcionar

## Guia

1. Decoradores
    - Anotação de tipos em funções
    - Checagem estática com decoradores
2. Classes
    - Classes de-sugar
    - Metaclasses
    - Método `__prepare__`
    - Dicionários personalizados
3. `__annotations__`
    - Lendo as anotações de tipo
    - Checagem estática
    - Instancecheck
4. Módulo typing
    - Checagem estática
5. Módulo forbidden fruit
    - Modificando attributos de tipos builtin
    - Ignorando invariantes do Python
    - Destruindo o interpretador
6. Ctypes
    - Modificando coisas proibidas
    - 42 == 666

## Decorador

In [109]:
def call_with(*args, **kwargs):
    def decorator(func):
        return func(*args, **kwargs)
    return decorator

In [110]:
@call_with()
def the_answer():
    return 42

the_answer

42

In [111]:
def add(x: int, y: int) -> int:
    return x + y

add.__annotations__

{'x': int, 'y': int, 'return': int}

In [112]:
def check_input(args, types):
    for x, tt in zip(args, types):
        if not isinstance(x, tt):
            raise TypeError(f'got {x.__class__.__name__}, expect {tt.__name__}')

def checked_result(value, tt):
    if not isinstance(value, tt):
        raise TypeError
    return value
            
    
def static(func):
    types = [func.__annotations__[k] for k in func.__code__.co_varnames]
    restype = func.__annotations__.get('result', object)

    def wrapped(*args):
        check_input(args, types)
        result = func(*args)
        return checked_result(result, restype)
    
    return wrapped

In [113]:
add_ = static(add)

In [114]:
add_(2, 2.2)

TypeError: got float, expect int

## Classes

In [115]:
class MyClass(object):
    cte = 42
    
    def f(self, x):
        return x % self.cte

In [116]:
attribs = {'cte': 42, 'f': lambda self, x: x % self.cte}
MyClass2 = type('MyClass2', (object,), attribs)

In [117]:
a = MyClass()
b = MyClass2()

In [118]:
a.f(100), b.f(100)

(16, 16)

In [119]:
from collections import Mapping

class Metaclass(type):
    def __new__(cls, name, bases, ns):
        ns = {k: v for k, v in ns.items() if not k.startswith('_')}
        return ns
        
    def __init__(self, name, bases, ns):
        print(f'Criando classe {name}')
        super().__init__(name, bases, ns)
        
    def __prepare__(name, bases):
        def set_variable(key, value):
            cls = annotations.get(key, object)
            
            if not isinstance(value, cls):
                raise TypeError(f'{key} is a {type(value)}, expect {cls}')
                
            if not key.startswith('_'):
                print(f'{key} = {value!r}')
        
        def set_annotation(key, value):
            print(f'{key}: {value!r}')
        
        ns = ObsDict(fset=set_variable)
        annotations = ns['__annotations__'] = ObsDict(fset=set_annotation)
        return ns
        

class ObsDict(dict):
    def __init__(self, fset=lambda k, v: None, fget=lambda k: None):
        self.fget = fget
        self.fset = fset
        
    def __setitem__(self, key, value):
        self.fset(key, value)
        dict.__setitem__(self, key, value)
        
    def __getitem__(self, key):
        self.fget(key)
        return dict.__getitem__(self, key)

In [120]:
@print
class Foo(metaclass=Metaclass):
    x:int = 1
    y:int = 1
    for _ in range(10):
        x, y = y, x + y
        
    result = y

x = 1
x: <class 'int'>
y = 1
y: <class 'int'>
x = 1
y = 2
x = 2
y = 3
x = 3
y = 5
x = 5
y = 8
x = 8
y = 13
x = 13
y = 21
x = 21
y = 34
x = 34
y = 55
x = 55
y = 89
x = 89
y = 144
result = 144
{'x': 89, 'y': 144, 'result': 144}


In [121]:
import sys
f = sys._getframe()


In [55]:
f.f_locals

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'class MyClass(object):\n    cte = 42\n    \n    def mod(self, x):\n        return x % self.cte',
  'class MyClass(object):\n    cte = 42\n    \n    def mod(self, x):\n        return x % self.cte',
  'class MyClass(object):\n    cte = 42\n    \n    def mod(self, x):\n        return x % self.cte',
  'class MyClass(object):\n    cte = 42\n    \n    def mod(self, x):\n        return x % self.cte',
  "attribs = {'cte': 42, 'mod': lambda self, x: x % self.cte}\nMyClass2 = type('MyClass2', (object,), attribs)",
  'a = MyClass()\nb = MyClass2()',
  'a.mod(100), b.mod(100)',
  "attribs = {'cte': 42, 'f': lambda self, x: x % self.cte}\nMyClass2 = type('MyClass2', (object,), attribs)",
  'class MyClass(object):\n    cte =

In [165]:
def add(x: int, y: int) -> int:
    return x + y

In [56]:
from typing import List

In [167]:
add_ = check_types(add)

In [57]:
isinstance([1,2,3], List[int])

TypeError: Subscripted generics cannot be used with class and instance checks

In [62]:
type(List[int]).mro()

[typing._GenericAlias, typing._Final, object]

In [64]:
isinstance(List[int], type)

False

In [70]:
meta = type(List[int])
List[int].__instancecheck__([1,2,3])

TypeError: Subscripted generics cannot be used with class and instance checks

In [104]:
def linstancecheck(lst_type, lst):
    tt = lst_type.__args__[0]
    return isinstance(lst, list) and all(isinstance(x, tt) for x in lst)

In [105]:
linstancecheck([1,2,3], List[int])

AttributeError: 'list' object has no attribute '__args__'

In [106]:
type(List).__instancecheck__ = linstancecheck

In [107]:
x.__args__[0]

int

In [108]:
isinstance([1,2,3], List[int])

True